# GROUP 35 - STAT 301 FINAL REPORT

### Group member:
- Doris Che (33996984)
- Eric Kim (87743738)
- Naufal Prasojo (14303440)
- Isaiah Nitin (79496493)


### INTRODUCTION
. . .

In [7]:
library(cowplot)
## In R console: install.packages("cowplot")  
## if not already installed 
library(tidyverse)
library(ggplot2)
library(dplyr)
library(broom)

# Read CSV files
athens_weekdays <- read.csv("data/athens_weekdays.csv")
athens_weekends <- read.csv("data/athens_weekends.csv")

# Check for missing values
colSums(is.na(athens_weekdays))
colSums(is.na(athens_weekends))

# Summary statistics
summary(athens_weekdays$realSum)
summary(athens_weekends$realSum)

# Cleaning data: removing nonsense outliers
"number obeservations for realSum >= 10000: "
nrow(athens_weekdays[athens_weekdays$realSum >= 10000, ])

athens_weekdays_clean <- athens_weekdays %>% filter(realSum < 10000)
summary(athens_weekdays_clean$realSum)

athens_weekends_clean <- athens_weekends %>% filter(realSum < 10000)
summary(athens_weekends_clean$realSum)

# Combine datasets for comparison plots + add binary period column
athens <- bind_rows(
  athens_weekdays_clean %>% mutate(period = "Weekday"),
  athens_weekends_clean %>% mutate(period = "Weekend")
) %>%
  mutate(period = factor(period, levels = c("Weekday","Weekend")))

head(athens)

X                    realSum 
                         0                          0 
                 room_type                room_shared 
                         0                          0 
              room_private            person_capacity 
                         0                          0 
         host_is_superhost                      multi 
                         0                          0 
                       biz         cleanliness_rating 
                         0                          0 
guest_satisfaction_overall                   bedrooms 
                         0                          0 
                      dist                 metro_dist 
                         0                          0 
                attr_index            attr_index_norm 
                         0                          0 
                rest_index            rest_index_norm 
                         0                          0 
                       lng                        lat 
                         0                          0

X                    realSum 
                         0                          0 
                 room_type                room_shared 
                         0                          0 
              room_private            person_capacity 
                         0                          0 
         host_is_superhost                      multi 
                         0                          0 
                       biz         cleanliness_rating 
                         0                          0 
guest_satisfaction_overall                   bedrooms 
                         0                          0 
                      dist                 metro_dist 
                         0                          0 
                attr_index            attr_index_norm 
                         0                          0 
                rest_index            rest_index_norm 
                         0                          0 
                       lng                        lat 
                         0                          0

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
   42.88    99.59   127.72   155.87   171.54 18545.45 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  42.88   98.66  127.72  147.58  171.54 1043.28 

[1] "number obeservations for realSum >= 10000: "

[1] 1

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  42.88   99.59  127.72  148.93  171.36 1043.28 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  42.88   98.66  127.72  147.58  171.54 1043.28 

,X,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,⋯,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,period
,<int>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0,129.82448,Entire home/apt,False,False,4,False,0,0,10,⋯,2,2.8139635,0.8818900,55.34857,2.086871,78.77838,5.915160,23.76600,37.98300,Weekday
2,1,138.96375,Entire home/apt,False,False,4,True,1,0,10,⋯,1,0.4072929,0.3045679,240.30665,9.060559,407.16770,30.572629,23.73168,37.97776,Weekday
3,2,156.30492,Entire home/apt,False,False,3,True,0,1,10,⋯,1,1.2372111,0.2884881,199.50737,7.522257,395.96740,29.731642,23.72200,37.97900,Weekday
4,3,91.62702,Entire home/apt,False,False,4,True,1,0,10,⋯,1,4.3674572,0.2974673,39.80305,1.500740,58.70658,4.408047,23.72712,38.01435,Weekday
5,4,74.05151,Private room,False,True,2,False,0,0,10,⋯,1,2.1941850,0.3852657,78.73340,2.968577,113.32597,8.509204,23.73391,37.99529,Weekday
6,5,113.88934,Entire home/apt,False,False,6,True,1,0,10,⋯,2,2.0712056,0.4538674,96.58899,3.641806,158.64432,11.911981,23.71584,37.98598,Weekday
